# 一、Condition对象
## 1. 基本介绍

Condition被称为条件变量，除了提供与Lock类似的acquire和release方法外，还提供了wait和notify方法。

`class threading.Condition(lock=None)` 

本类用于实现条件变量对象。条件变量对象允许多条线程保持等待状态直到接收另一条线程的通知。 
如果选择传入 lock 参数，只能使用 Lock 或 RLock对象，而且它会被当做一个隐性锁使用。如果不传此参数，那么程序会自动隐性地创建一个 RLock 对象。

`acquire(*args)`  

本方法用于获取隐性锁（关联锁），它调用隐性锁的 acquire() 方法，并返回其所返回的值

`release()`  

同上，本方法无返回值

`wait(timeout=None)`  

等待通知或超时。如果线程没有获取到锁就调用了此方法，那么将引发 RuntimeError 异常   
本方法会释放隐性锁，然后阻塞直到被其他线程的调用此条件变量的 notify() 或 notify_all() 唤醒，或超时。一旦被唤醒或超时，该线程将立即重新获取锁并返回  
timeout 参数是以秒为单位的浮点数   
如果隐性锁是一个 RLock 对象，因为调用它的 release() 方法未必能够释放该锁，所以本方法会使用 RLock 对象的一个内部接口，该接口可以立即释放多重迭代的 RLock 锁。并且在需要重新获取锁的时候，也会使用一个类似的内部接口来恢复多重的迭代级别  
本方法所阻塞的线程如果是被唤醒的，那么本方法会返回一个 True，如果是超时了，则返回 False  

`notify(n=1)`  

本方法默认用于唤醒处于等待本条件变量的线程。如果调用本方法的线程并没有获得锁，将引发 RuntimeError 异常  
本方法至多可唤醒所有正在等待本条件变量的线程中的 n 个。如果调用时没有线程处于等待操作，那么本方法的调用是一个空操作  
现在版本对本方法的实现为：在有足够多处于等待状态的线程的条件下，本方法将正好唤醒其中的 n 个，而不是像上一条中讲的“至多 n 个”。不过这种行为并不可靠。在将来，本方法很可能偶尔唤醒超过 n 条线程  

`notify_all()`  

唤醒正在等待本条件变量的所有线程。
acquire与release可以用with语句代替，比如：

```
with lock_con:
    lock_con.wait()
```

# 二、简单例子
## 1.两个线程轮流挂起与激活

In [1]:
import threading
import time


def fun(cndition):
    time.sleep(1)  # 确保先运行t2
    # 获得锁
    cndition.acquire()
    print('thread1 acquires lock.')
    # 唤醒t2
    cndition.notify()
    # 进入等待状态，等待其他线程唤醒
    cndition.wait()
    print('thread1 acquires lock again.')
    # 释放锁
    cndition.release()


def fun2(cndition):
    # 获得锁
    cndition.acquire()
    print('thread2 acquires lock.')
    # 进入等待状态，等待其他线程唤醒
    cndition.wait()
    print('thread2 acquires lock again.')
    # 唤醒t1
    cndition.notify()
    # 释放锁
    cndition.release()


if __name__ == '__main__':
    cndition = threading.Condition()
    t1 = threading.Thread(target=fun, args=(cndition,))
    t2 = threading.Thread(target=fun2, args=(cndition,))
    t1.start()
    t2.start()

thread2 acquires lock.
thread1 acquires lock.
thread2 acquires lock again.
thread1 acquires lock again.


## 2.实现生产与消费者模式

In [ ]:
import threading, time
from random import randint


class Producer(threading.Thread):
    def run(self):
        global L
        while True:
            val = randint(0, 100)
            # if lock_con.acquire():
            #     L.append(val)
            #     print(f"生产者:{self.name}, Append:{val},队列：{L}")
            #     lock_con.notify()
            #     lock_con.release()
            with lock_con:
                L.append(val)
                print(f"生产者:{self.name}, Append:{val}, L = {L}")
                lock_con.notify()
            time.sleep(3)


class Consumer(threading.Thread):
    def run(self):
        global L
        while True:
            with lock_con:
                if len(L) == 0:
                    print("队列为空，请等待。。。")
                    lock_con.wait()
                print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
                print(f"消费者: {self.name}, Delete: {L[0]}")
                del L[0]
            time.sleep(0.5)


if __name__ == '__main__':
    L = []  # 消费物队列
    lock_con = threading.Condition()
    threads = []
    # 若干个生产者线程
    for i in range(3):
        threads.append(Producer())
    threads.append(Consumer())
    for t in threads:
        t.start()
    for t in threads:
        t.join()

生产者:Thread-7, Append:91, L = [91]
生产者:Thread-8, Append:64, L = [91, 64]
生产者:Thread-9, Append:71, L = [91, 64, 71]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 91
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 64
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 71
队列为空，请等待。。。
生产者:Thread-9, Append:93, L = [93]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 93
生产者:Thread-8, Append:65, L = [65]
生产者:Thread-7, Append:3, L = [65, 3]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 65
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 3
队列为空，请等待。。。
生产者:Thread-8, Append:55, L = [55]
生产者:Thread-7, Append:35, L = [55, 35]
生产者:Thread-9, Append:27, L = [55, 35, 27]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 55
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 35
>>>>>

队列为空，请等待。。。
生产者:Thread-8, Append:79, L = [79]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 79
生产者:Thread-9, Append:36, L = [36]
生产者:Thread-7, Append:26, L = [36, 26]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 36
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 26
队列为空，请等待。。。
生产者:Thread-8, Append:50, L = [50]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 50
生产者:Thread-9, Append:86, L = [86]
生产者:Thread-7, Append:73, L = [86, 73]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 86
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 73
队列为空，请等待。。。
生产者:Thread-8, Append:74, L = [74]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 74
生产者:Thread-9, Append:6, L = [6]
生产者:Thread-7, Append:73, L = [6, 73]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
消费者: Thread-10, Delete: 6
>>>>>>>>>>